In [ ]:
from energieapp.common_imports import *
show_home_button()
from energieapp.db_connection import get_engine
engine = get_engine()

In [ ]:
DEFAULT_MANAGED_BY  = 10046
DEFAULT_INSERTED_BY = 10046

class ReplacementManager:
    def __init__(self, engine):
        self.engine = engine    

    def _connect(self):
        conn = self.engine.raw_connection()
        conn.autocommit = False        
        return conn

    def _resolve_ids(self, conn, oude_desc, nieuwe_desc):
        cur = conn.cursor()
        cur.execute("SELECT Id FROM dbo.TBL_Registrator WHERE [Description]=?", oude_desc)
        row = cur.fetchone()
        if not row:
            raise ValueError(f"Oude registrator '{oude_desc}' niet gevonden.")
        oude_id = row[0]

        cur.execute("SELECT Id FROM dbo.TBL_Registrator WHERE [Description]=?", nieuwe_desc)
        row = cur.fetchone()
        if row:
            nieuwe_id = row[0]
        else:
            cur.execute(
                "INSERT INTO dbo.TBL_Registrator ([Description], ManagedBy, InsertedBy) "
                "VALUES (?, ?, ?)",
                nieuwe_desc, DEFAULT_MANAGED_BY, DEFAULT_INSERTED_BY
            )
            cur.execute("SELECT CAST(SCOPE_IDENTITY() AS int)")
            nieuwe_id = cur.fetchone()[0]

        cur.execute("SELECT Id FROM dbo.TBL_Meter WHERE RegistratorId=?", nieuwe_id)
        row = cur.fetchone()
        meter_id = row[0] if row else None
        return oude_id, nieuwe_id, meter_id

    def _select_overview(self, conn, oude_id, nieuwe_id):
        sql = text("""
            SELECT DISTINCT
              re.Id   AS RegisterId,
              re.[Status] AS Status,
              rrt.AddressId AS Meter_Telwerk,
              re.Address,
              re.RegistratorId      AS Register_OpnemerId,
              re.[Description]      AS Register,
              m.Id                 AS MeterId,
              m.RegistratorId      AS Meter_RegistratorId,
              rg.Id                AS OpnemerId,
              rg.[Description]     AS Opnemer_Description,
              rg.ManagedBy,
              dl.Active,
              dl.ProductionStatus
            FROM dbo.TBL_Register re
            INNER JOIN dbo.TBL_Meter m
              ON m.RegistratorId = re.RegistratorId
            INNER JOIN dbo.TBL_Registrator rg
              ON rg.Id = re.RegistratorId
            INNER JOIN dbo.TBL_Dialerlist dl
              ON dl.RegistratorId = rg.Id
            LEFT JOIN dbo.TBL_Register_ReadingType rrt
              ON rrt.RegisterId = re.Id
            WHERE re.RegistratorId IN (:oid, :nid)
            ORDER BY re.RegistratorId, re.Id;
        """)
        return pd.read_sql(sql, self.engine, params={"oid": oude_id, "nid": nieuwe_id})

    def _select_registrator_info(self, conn, oude_id, nieuwe_id):
        sql = text("""
            SELECT DISTINCT
              rg.Id                 AS RegistratorId,
              rg.[Description]      AS Registrator_Description,
              rg.ManagedBy,
              dl.Active,
              dl.ProductionStatus,
              ra.Id                 AS Alternative_Protocol
            FROM dbo.TBL_Registrator rg
            INNER JOIN dbo.TBL_Dialerlist dl
              ON dl.RegistratorId = rg.Id
            LEFT  JOIN dbo.TBL_Registrator_alternativeprotocol ra
              ON ra.RegistratorId = rg.Id
            WHERE rg.Id IN (:oid, :nid)
            ORDER BY rg.Id;
        """)
        return pd.read_sql(sql, self.engine, params={"oid": oude_id, "nid": nieuwe_id})

    def _select_waiting(self, conn, nieuwe_id):
        sql = text("""
            SELECT *
            FROM dbo.TBL_clientqueue
            WHERE RegistratorId = :nid
              AND [State] = 'WAITING';
        """)
        return pd.read_sql(sql, self.engine, params={"nid": nieuwe_id})

    # --------------------------- updates ----------------------------
    def _apply_updates(self, conn, oude_id, nieuwe_id, meter_id, oude_desc, nieuwe_desc):
        cur = conn.cursor()
        cur.execute(
            "UPDATE dbo.TBL_Register SET RegistratorId=? "
            "WHERE RegistratorId=? AND [Status]='active'",
            nieuwe_id, oude_id
        )
        if meter_id is not None:
            cur.execute(
                "UPDATE dbo.TBL_Register SET MeterId=? "
                "WHERE RegistratorId=? AND [Status]='active'",
                meter_id, nieuwe_id
            )
        old_prod_status = f"{oude_desc} (Vervangen door {nieuwe_desc})"
        cur.execute(
            "UPDATE dbo.TBL_Dialerlist SET Active='n', ProductionStatus=? "
            "WHERE RegistratorId=?",
            old_prod_status, oude_id
        )
        cur.execute(
            "UPDATE dbo.TBL_Dialerlist SET Active='j', ProductionStatus=NULL "
            "WHERE RegistratorId=?", nieuwe_id
        )
        cur.execute(
            "UPDATE dbo.TBL_Registrator SET [Description]=?, ManagedBy=? WHERE Id=?",
            old_prod_status, DEFAULT_MANAGED_BY, oude_id
        )
        new_desc_full = f"{nieuwe_desc} (Vervanging {oude_desc})"
        cur.execute(
            "UPDATE dbo.TBL_Registrator SET [Description]=? WHERE Id=?",
            new_desc_full, nieuwe_id
        )
        cur.execute(
            "DELETE FROM dbo.TBL_clientqueue "
            "WHERE RegistratorId=? AND [State]='WAITING'",
            nieuwe_id
        )

    # --------------------------- API -------------------------------
    def compare(self, oude_desc: str, nieuwe_desc: str):
        conn = self._connect()
        try:
            oid, nid, mid   = self._resolve_ids(conn, oude_desc, nieuwe_desc)
            bef_regs        = self._select_overview(conn, oid, nid)
            bef_regsio      = self._select_registrator_info(conn, oid, nid)
            bef_waiting     = self._select_waiting(conn, nid)

            self._apply_updates(conn, oid, nid, mid, oude_desc, nieuwe_desc)

            aft_regs        = self._select_overview(conn, oid, nid)
            aft_regsio      = self._select_registrator_info(conn, oid, nid)
            aft_waiting     = self._select_waiting(conn, nid)

            conn.rollback()          # niets opslaan in compare-modus
            return bef_regs, bef_regsio, bef_waiting, aft_regs, aft_regsio, aft_waiting
        finally:
            conn.close()

    def execute(self, oude_desc: str, nieuwe_desc: str):
        conn = self._connect()
        try:
            oid, nid, mid = self._resolve_ids(conn, oude_desc, nieuwe_desc)
            self._apply_updates(conn, oid, nid, mid, oude_desc, nieuwe_desc)
            conn.commit()            # definitief wegschrijven
            regs    = self._select_overview(conn, oid, nid)
            regio   = self._select_registrator_info(conn, oid, nid)
            waiting = self._select_waiting(conn, nid)
            return regs, regio, waiting
        except Exception:
            conn.rollback()
            raise
        finally:
            conn.close()

rm = ReplacementManager(engine)

In [ ]:
class ReplacementUI:
    def __init__(self, conn_str):
        self.manager = ReplacementManager(conn_str)
        self.out = widgets.Output(layout={'border':'1px solid #ccc','padding':'10px'})
        self.oude_input = widgets.Text(placeholder="Oude Registrator Description", description="Oude:")
        self.nieuwe_input = widgets.Text(placeholder="Nieuwe Registrator Description", description="Nieuwe:")
        self.compare_btn = widgets.Button(description="Vergelijk Voor/Na", button_style='info')
        self.exec_btn    = widgets.Button(description="Voer Door", button_style='danger', disabled=True)
        self.compare_btn.on_click(self._on_compare)
        self.exec_btn.on_click(self._on_execute)
        self.tabs = widgets.Tab()
        header = widgets.HBox([self.oude_input, self.nieuwe_input, self.compare_btn, self.exec_btn])
        self.ui = widgets.VBox([widgets.HTML("<b>Vervangings-Tool</b>"), header, self.out, self.tabs])

    def display(self):
        display(self.ui)

    def _make_grid(self, df):
        cols = df.columns.tolist()
        columnDefs = [
            {**{'headerName': c, 'field': c},
             **({'pinned': 'left', 'width': 100} if i == 0 else {})}
            for i, c in enumerate(cols)
        ]
        grid_options = {
            'enableSorting': True,
            'enableFilter': True,
            'pagination': True,
            'paginationPageSize': 20,
            'columnDefs': columnDefs,
            'defaultColDef': {'resizable': True, 'autoSize': True}
        }
        grid = Grid(grid_data=df, grid_options=grid_options)
        grid.layout = widgets.Layout(width='100%', height='300px')
        return grid

    def _on_compare(self, _):
        with self.out:
            clear_output()
        self.exec_btn.disabled = True
        oude, nieuwe = self.oude_input.value.strip(), self.nieuwe_input.value.strip()
        if not oude or not nieuwe:
            with self.out:
                print("Fout: beide velden moeten ingevuld zijn.")
            return
        try:
            bef_regs, bef_regsio, bef_waiting, aft_regs, aft_regsio, aft_waiting = \
                self.manager.compare(oude, nieuwe)
        except Exception as e:
            with self.out:
                print(f"Fout bij vergelijken: {e}")
            return

        g_bef_regs    = self._make_grid(bef_regs)
        g_aft_regs    = self._make_grid(aft_regs)
        g_bef_regsio  = self._make_grid(bef_regsio)
        g_aft_regsio  = self._make_grid(aft_regsio)
        g_bef_waiting = self._make_grid(bef_waiting)
        g_aft_waiting = self._make_grid(aft_waiting)

        regs_tab = widgets.VBox([
            widgets.HTML("<b>Registers: Voor</b>"), g_bef_regs,
            widgets.HTML("<b>Registers: Na</b>"),  g_aft_regs
        ])
        regsio_tab = widgets.VBox([
            widgets.HTML("<b>Opnemers: Voor</b>"), g_bef_regsio,
            widgets.HTML("<b>Opnemers: Na</b>"),  g_aft_regsio
        ])
        waiting_tab = widgets.VBox([
            widgets.HTML("<b>Wachtende jobs: Voor</b>"), g_bef_waiting,
            widgets.HTML("<b>Wachtende jobs: Na</b>"),  g_aft_waiting
        ])

        self.tabs.children = [regs_tab, regsio_tab, waiting_tab]
        self.tabs.set_title(0, "Registers")
        self.tabs.set_title(1, "Opnemers")
        self.tabs.set_title(2, "Wachtende jobs")

        with self.out:
            print("Vergelijking gereed. Klik op 'Voer Door' om definitief uit te voeren.")
        self.exec_btn.disabled = False

    def _on_execute(self, _):
        with self.out:
            clear_output()
        oude, nieuwe = self.oude_input.value.strip(), self.nieuwe_input.value.strip()
        try:
            regs, regio, waiting = self.manager.execute(oude, nieuwe)
        except Exception as e:
            with self.out:
                print(f"Fout bij uitvoeren: {e}")
            return

        g_regs    = self._make_grid(regs)
        g_regio   = self._make_grid(regio)
        g_waiting = self._make_grid(waiting)

        regs_tab = widgets.VBox([widgets.HTML("<b>Definitieve Registers</b>"), g_regs])
        regio_tab = widgets.VBox([widgets.HTML("<b>Definitieve Opnemers</b>"), g_regio])
        waiting_tab = widgets.VBox([widgets.HTML("<b>Definitieve Wachtende jobs</b>"), g_waiting])

        self.tabs.children = [regs_tab, regio_tab, waiting_tab]
        self.tabs.set_title(0, "Registers")
        self.tabs.set_title(1, "Opnemers")
        self.tabs.set_title(2, "Wachtende jobs")

        with self.out:
            print("Wijzigingen doorgevoerd en opgeslagen.")

In [ ]:
ui = ReplacementUI(engine)
ui.display()